# <div align="center" style="color: #ff5733;">New Pos Addition and Mapping</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\PosAdd_20240705.csv")

In [ ]:
df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data May042024.xlsx", sheet_name = 'Sheet1')
df.head(10)

In [3]:
df.head()

,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM,Brand
0,Xtreme Appliances Batangas,Standalone,POS Operational,"April 14, 2024",MFW9BYL,South Luzon 3,Jan jan Dela Cruz,Xtreme
1,Workstation Home Appliances Ayala By The Bay,Ayala Malls Manila Bay,POS Operational,"May 10, 2024",M7V4Z2N,Grace Belarmino,Grace Belarmino,Workstation
2,SHAR Novaliches,Standalone,POS Operational,"June 26, 2024",M0C5496,Sander Maghirang,Sander Maghirang,JSL
3,Realme Serin,Ayala Mall Serin,POS Operational,"June 1, 2024",M87F26K,Aubrey Lalong Isip,Aubrey Lalong Isip,Rulls
4,Homeworks Binan,Central Mall Binan,POS Operational,"May 30, 2024",MQ5R16Y,Aubrey Lalong Isip,Kenneth Mendoza,Homeworks


In [4]:
df.rename(columns = {'Brand': 'BrandStores'}, inplace = True)
df.columns

Index(['POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY',
       'REGION', 'TSM', 'BrandStores'],
      dtype='object')

In [5]:
df['BrandStores'].value_counts()

BrandStores
Home Along     3
Homeworks      2
Xtreme         1
Workstation    1
JSL            1
Rulls          1
Name: count, dtype: int64

In [6]:
df['PURPLE KEY'].value_counts()

PURPLE KEY
MFW9BYL    1
M7V4Z2N    1
M0C5496    1
M87F26K    1
MQ5R16Y    1
MVZS5FA    1
MTKYL7W    1
MHDTX6A    1
MWN9Y1D    1
Name: count, dtype: int64

In [7]:
df = df[['BrandStores', 'POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION', 'TSM']]
df

,BrandStores,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM
0,Xtreme,Xtreme Appliances Batangas,Standalone,POS Operational,"April 14, 2024",MFW9BYL,South Luzon 3,Jan jan Dela Cruz
1,Workstation,Workstation Home Appliances Ayala By The Bay,Ayala Malls Manila Bay,POS Operational,"May 10, 2024",M7V4Z2N,Grace Belarmino,Grace Belarmino
2,JSL,SHAR Novaliches,Standalone,POS Operational,"June 26, 2024",M0C5496,Sander Maghirang,Sander Maghirang
3,Rulls,Realme Serin,Ayala Mall Serin,POS Operational,"June 1, 2024",M87F26K,Aubrey Lalong Isip,Aubrey Lalong Isip
4,Homeworks,Homeworks Binan,Central Mall Binan,POS Operational,"May 30, 2024",MQ5R16Y,Aubrey Lalong Isip,Kenneth Mendoza
5,Home Along,Home Along Floridablanca,Standalone,POS Operational,"June 21, 2024",MVZS5FA,Rachelle Belmonte,Rachelle Belmonte
6,Home Along,Home Along Bongabon,Standalone,POS Operational,"June 7, 2024",MTKYL7W,Rachelle Belmonte,Rachelle Belmonte
7,Home Along,Home Along Balanga 1,Standalone,POS Operational,"June 13, 2024",MHDTX6A,Rachelle Belmonte,Rachelle Belmonte
8,Homeworks,Event Store Homeworks Commonwealth Atrium May ...,Standalone,Closed,"May 20, 2024",MWN9Y1D,Sander Maghirang,NCR North


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BrandStores      9 non-null      object
 1   POS NAME         9 non-null      object
 2   STORE LOCATION   9 non-null      object
 3   STATUS           9 non-null      object
 4   DATE ACTIVATION  9 non-null      object
 5   PURPLE KEY       9 non-null      object
 6   REGION           9 non-null      object
 7   TSM              9 non-null      object
dtypes: object(8)
memory usage: 704.0+ bytes


In [9]:
df.rename(columns={'POS NAME':'POS', 'DATE ACTIVATION':'DateOpened', 'STORE LOCATION': 'StoreLocation', 'STATUS':'Status', 'PURPLE KEY':'PurpleKey', 'REGION':'Region'}, inplace = True)

In [10]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%B %d, %Y')


In [11]:
print(df['DateOpened'].dtype)
# datetime64[ns]
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

datetime64[ns]
2024-04-14 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BrandStores    9 non-null      object
 1   POS            9 non-null      object
 2   StoreLocation  9 non-null      object
 3   Status         9 non-null      object
 4   DateOpened     9 non-null      object
 5   PurpleKey      9 non-null      object
 6   Region         9 non-null      object
 7   TSM            9 non-null      object
dtypes: object(8)
memory usage: 704.0+ bytes


In [12]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%b-%y')

ValueError: time data "2024-04-14" doesn't match format "%d-%b-%y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
print(df['DateOpened'].dtype)
# datetime64[ns]

In [13]:
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00

2024-04-14


In [14]:
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

AttributeError: Can only use .dt accessor with datetimelike values

In [15]:
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%Y-%m-%d')

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BrandStores    9 non-null      object        
 1   POS            9 non-null      object        
 2   StoreLocation  9 non-null      object        
 3   Status         9 non-null      object        
 4   DateOpened     9 non-null      datetime64[ns]
 5   PurpleKey      9 non-null      object        
 6   Region         9 non-null      object        
 7   TSM            9 non-null      object        
dtypes: datetime64[ns](1), object(7)
memory usage: 704.0+ bytes


In [17]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Xtreme,Xtreme Appliances Batangas,Standalone,POS Operational,2024-04-14,MFW9BYL,South Luzon 3,Jan jan Dela Cruz
1,Workstation,Workstation Home Appliances Ayala By The Bay,Ayala Malls Manila Bay,POS Operational,2024-05-10,M7V4Z2N,Grace Belarmino,Grace Belarmino
2,JSL,SHAR Novaliches,Standalone,POS Operational,2024-06-26,M0C5496,Sander Maghirang,Sander Maghirang
3,Rulls,Realme Serin,Ayala Mall Serin,POS Operational,2024-06-01,M87F26K,Aubrey Lalong Isip,Aubrey Lalong Isip
4,Homeworks,Homeworks Binan,Central Mall Binan,POS Operational,2024-05-30,MQ5R16Y,Aubrey Lalong Isip,Kenneth Mendoza


In [18]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
try:
    d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
    print("Table still exists")
except:
    print("Table dropped")

Query is running:   0%|          |Table dropped


In [19]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1
as 
select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;
"""
client.query(sq)



QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8bee7bdf-26b3-44c2-acc5-4ed574cd10e0>

In [20]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of d is:\t{d.shape}")

Job ID b02c22bc-1ade-403d-913d-061b86883f59 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of d is:	(678, 8)


In [21]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""

dfsil = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID efc5e2c1-48c7-4702-9443-5dc4c9626456 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [22]:
dfsil.BrandStores.value_counts().sort_index()

BrandStores
A2RJ                  3
Affordables           2
Ansons               15
ArkHome               4
Automatic Centre     11
Dizon Furniture       1
ESLA                 51
FC Home             109
Faith Glorious        9
Gadgetmate           10
Hardware Sugar        2
Home Along           91
Home Factory          2
Homeworks            13
JSL                   6
JTG                   1
Jhunpyo               8
Junliz                8
KUM Home Center       3
Kservico             10
MemoXpress           54
PHPS                  2
Power Mac Center      5
Presnet              21
Prince and Aviel      3
RCB Mobile            3
Rosa Mobile           9
Rulls               161
Sogo Home             3
Willy and Sons       29
Workstation           6
Wyse                  1
Xtreme               22
Name: count, dtype: int64

In [23]:
dfsil.shape

(678, 8)

In [24]:
# Set table_id to the ID of the table to append to.
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 687 rows and 8 columns to prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1


In [25]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is: {df.shape}")

Job ID c9c0f572-bd0d-4926-9f6d-07e10c4bf881 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (687, 8)


In [26]:
df.tail(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
677,Xtreme,Xtreme Appliances Calamba,Carmel Mall,POS Operational,2023-10-15,MUBPW04,South Luzon 2,Christian Klide Alberto
678,Homeworks,Event Store Homeworks Commonwealth Atrium May ...,Standalone,Closed,2024-05-20,MWN9Y1D,Sander Maghirang,NCR North
679,Xtreme,Xtreme Appliances Batangas,Standalone,POS Operational,2024-04-14,MFW9BYL,South Luzon 3,Jan jan Dela Cruz
680,JSL,SHAR Novaliches,Standalone,POS Operational,2024-06-26,M0C5496,Sander Maghirang,Sander Maghirang
681,Home Along,Home Along Floridablanca,Standalone,POS Operational,2024-06-21,MVZS5FA,Rachelle Belmonte,Rachelle Belmonte
682,Home Along,Home Along Bongabon,Standalone,POS Operational,2024-06-07,MTKYL7W,Rachelle Belmonte,Rachelle Belmonte
683,Home Along,Home Along Balanga 1,Standalone,POS Operational,2024-06-13,MHDTX6A,Rachelle Belmonte,Rachelle Belmonte
684,Rulls,Realme Serin,Ayala Mall Serin,POS Operational,2024-06-01,M87F26K,Aubrey Lalong Isip,Aubrey Lalong Isip
685,Homeworks,Homeworks Binan,Central Mall Binan,POS Operational,2024-05-30,MQ5R16Y,Aubrey Lalong Isip,Kenneth Mendoza
686,Workstation,Workstation Home Appliances Ayala By The Bay,Ayala Malls Manila Bay,POS Operational,2024-05-10,M7V4Z2N,Grace Belarmino,Grace Belarmino


In [27]:
df['BrandStores'].value_counts().sort_index()

BrandStores
A2RJ                  3
Affordables           2
Ansons               15
ArkHome               4
Automatic Centre     11
Dizon Furniture       1
ESLA                 51
FC Home             109
Faith Glorious        9
Gadgetmate           10
Hardware Sugar        2
Home Along           94
Home Factory          2
Homeworks            15
JSL                   7
JTG                   1
Jhunpyo               8
Junliz                8
KUM Home Center       3
Kservico             10
MemoXpress           54
PHPS                  2
Power Mac Center      5
Presnet              21
Prince and Aviel      3
RCB Mobile            3
Rosa Mobile           9
Rulls               162
Sogo Home             3
Willy and Sons       29
Workstation           7
Wyse                  1
Xtreme               23
Name: count, dtype: int64

In [28]:
# Find duplicate values in 'POS' column
duplicates = df[df.duplicated('POS', keep=False)]

# Print the entire rows for duplicate entries in 'POS'
if not duplicates.empty:
    print("Duplicate values in 'POS' column:")
    duplicates.to_csv("Temp.csv")
else:
    print("No duplicate values found in 'POS' column.")
    
dd = pd.read_csv("Temp.csv")
dd.sort_values(by='POS')

Duplicate values in 'POS' column:


,Unnamed: 0,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,2,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,M9ZK6IU,NaN,NaN
1,3,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,MV4UO0X,NaN,NaN
14,642,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MN3GL66,NCR North 2,Ben Antonio Lacsamana
13,640,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MGP2AQI,NCR North,Ben Antonio Lacsamana
15,647,Home Along,Home Along Malanday,NaN,NaN,2022-07-11,MURFAIV,NCR North,Ben Antonio Lacsamana
16,648,Home Along,Home Along Malanday,Standalone,POS Operational,2022-07-11,MNBW4RZ,NCR North,Ben Antonio Lacsamana
5,560,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MDGYHQJ,NCR South,Klein Justine Tamayo
6,561,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MU82X9Z,NCR South,Klein Justine Tamayo
12,579,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MDO3KI4,NCR South,Klein Justine Tamayo
11,578,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MRGP0WQ,NCR South,Klein Justine Tamayo


In [29]:
pd.set_option('display.max_rows', None)
# Count the occurrences of each value in the 'PurpleKey' column
purple_key_counts = df['PurpleKey'].value_counts()

# Filter to include only values with count >= 2
filtered_counts = purple_key_counts[purple_key_counts >= 2]

# Display the filtered counts
print(filtered_counts)


Series([], Name: count, dtype: int64)


In [30]:
df[df['PurpleKey']== 'MFW9BYL']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
679,Xtreme,Xtreme Appliances Batangas,Standalone,POS Operational,2024-04-14,MFW9BYL,South Luzon 3,Jan jan Dela Cruz


In [31]:
sq = """drop table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e7479425-62a1-490d-a577-d498172a66df>

In [32]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING as select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ea7cc9c1-2bba-48a3-8d7e-5c2424b5a11b>

In [33]:
client.query("commit")

QueryJob<project=prj-prod-dataplatform, location=US, id=5bac2c52-27e1-4ca1-9a4b-2600a59b50dc>

In [34]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.shape

Job ID c1c392d6-60fe-4785-8c00-c9c7dd0dcfd8 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(687, 8)

In [35]:
df[df['BrandStores']=='RCB Mobile'].sort_values(by='DateOpened')

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
117,RCB Mobile,K Multibrand Binakayan,Standalone,POS Operational,2024-06-28,MKGZCLU,South Luzon 1 (B),Ricky Sarayno
118,RCB Mobile,RCB Multibrand Lotus Mall,Lotus Mall Imus,POS Operational,2024-06-28,M39HE4Y,South Luzon 1 (B),Ricky Sarayno
119,RCB Mobile,Regie C Basillo Lotus Mall Imus,Lotus Mall Imus,POS Operational,2024-06-28,M01FEG9,South Luzon 1 (B),Ricky Sarayno


In [36]:
df.head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Balibago,LOT 5 BLK 9 GARNET ST. RHINELAND COMMERCIAL CO...,Operational,2023-08-01,MOG243Y,South Luzon 2,Klide Alberto
1,Homeworks,Event Store Homeworks Commonwealth Atrium May ...,Standalone,Closed,2024-05-20,MWN9Y1D,Sander Maghirang,NCR North
2,Xtreme,Xtreme Appliances Anabu,None,None,2022-08-27,MR2YDE8,None,None
3,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,M9ZK6IU,None,None
4,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,MV4UO0X,None,None
5,Home Along,Home Along Sumulong,None,None,2022-07-12,M2G78K4,None,None
6,Dizon Furniture,Dizon Furniture Tiendesitas,None,None,2022-07-23,MOE3JQH,None,None
7,Xtreme,Xtreme Appliances Lucky Gold Plaza,Lucky Gold Plaza,For Opening,2024-04-14,MO3AHGW,NCR East,Albee David
8,MemoXpress,Memoxpress Robinsons Place Antipolo,Robinsons Antipolo,For Opening,2024-04-15,MS1L6R8,NCR East,Albee David
9,MemoXpress,Memoxpress Ayala Marikina,Ayala Malls Marikina,For Opening,2024-04-15,MOVZDHU,NCR East,Albee David


In [37]:
df.sort_values('DateOpened', ascending=False)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
117,RCB Mobile,K Multibrand Binakayan,Standalone,POS Operational,2024-06-28,MKGZCLU,South Luzon 1 (B),Ricky Sarayno
118,RCB Mobile,RCB Multibrand Lotus Mall,Lotus Mall Imus,POS Operational,2024-06-28,M39HE4Y,South Luzon 1 (B),Ricky Sarayno
119,RCB Mobile,Regie C Basillo Lotus Mall Imus,Lotus Mall Imus,POS Operational,2024-06-28,M01FEG9,South Luzon 1 (B),Ricky Sarayno
251,JSL,JSL Waltermart Munoz,Waltermart Munoz,For Opening,2024-06-26,M7P1GIK,NCR North,Sander Maghirang
246,JSL,SHAR Novaliches,Standalone,POS Operational,2024-06-26,M0C5496,Sander Maghirang,Sander Maghirang
250,JSL,AS Sabayo Robinsons Novaliches,Robinsons Novaliches,For Opening,2024-06-26,MZITK84,NCR North,Sander Maghirang
249,JSL,AS Sabayo Wilcon Visayas Ave,Standalone,For Opening,2024-06-26,MYPG7T9,NCR North,Sander Maghirang
248,JSL,SHAR Novaliches,Standalone,For Opening,2024-06-26,MDZSR97,NCR North,Sander Maghirang
247,JSL,JSL Novaliches,Standalone,For Opening,2024-06-26,M65WJCU,NCR North,Sander Maghirang
252,JSL,ABLNR iMall,iMall Caloocan,For Opening,2024-06-26,MC08YM1,NCR North,Sander Maghirang


In [38]:
df[df['PurpleKey'] =='MFW9BYL']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
261,Xtreme,Xtreme Appliances Batangas,Standalone,POS Operational,2024-04-14,MFW9BYL,South Luzon 3,Jan jan Dela Cruz


In [39]:
df[df['BrandStores']=='RCB Mobile']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
117,RCB Mobile,K Multibrand Binakayan,Standalone,POS Operational,2024-06-28,MKGZCLU,South Luzon 1 (B),Ricky Sarayno
118,RCB Mobile,RCB Multibrand Lotus Mall,Lotus Mall Imus,POS Operational,2024-06-28,M39HE4Y,South Luzon 1 (B),Ricky Sarayno
119,RCB Mobile,Regie C Basillo Lotus Mall Imus,Lotus Mall Imus,POS Operational,2024-06-28,M01FEG9,South Luzon 1 (B),Ricky Sarayno


In [40]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data.xlsx", index = False)